This tutorial will teach you how to transform a docx into a pandas dataframe and then save it as a CSV file.

Prerequisites:
* you will need Python installed on your computer
* make sure that your file is a .docx and not a .doc

## Install the appropriate Python packages

In [61]:
#specific to extracting information from word documents
import os
import zipfile
#other tools useful in extracting the information from our document
import re
import copy


#to read XML and JSON
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import XML
from lxml import etree
import json

#to pretty print our xml:
import xml.dom.minidom
from xml.dom.minidom import parse, parseString

#to use dataframes
import pandas as pd

#import dateparser
#import collections


## Anatomy of a .docx file

What is a docx file? Well, it is in fact an archive of xml files!
Let's look at it.

- Rename the file extension from .docx to .zip 
- Unzip the newly renamed file

In [62]:
#file = "burmese/burmese_selection"
#file = "turkish/turkish_sample"
file = "india/india_sample"

docxFileName = file+".docx"
docxZip = zipfile.ZipFile(docxFileName)
documentXML = docxZip.read('word/document.xml')
stylesXML = docxZip.read('word/styles.xml')
et = etree.XML(documentXML)
ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}

If you want to see what are the XML files hidden in your docx file you can use: `namelist()`

In [63]:
#docxZip.namelist()

In [64]:
#Title
#ReferenceOld
#ReferenceNew
#Item date
#Content description
#Physical description
#Creator
#Language
'''
Is it possible to use tags multiple times for each entry / row? 
Fields like language and creator potentially have multiple entries. 
It would also be useful if possible to have multiple reference tags, e.g. ReferenceOld, ReferenceNew. 
(in the example you provided there are actually 2 references given, e.g. 1 Mss Eur G 1 is technically 
ReferenceOld: 1, ReferenceNew: Mss Eur G 1), or a reference which identifies a new record, 
and a legacy reference held by that record. If not then these could be recorded as one tag 
and split by the cataloguer at a later stage.

 

The International Standard for Archival Description specifies 6 essential fields
to support international data transfer: reference, title, creator, dates, extent, 
and level of description. Extent and level are best added after by the cataloguer 
and not taken from the legacy source.

 

Similarly, the IAMS guidelines specify 15/17 minimum fields depending on level of
description, but many of them are auto-populated (e.g. legal status), inherited from
collection descriptions, or best completed with automated look-up (e.g. ISO language and script codes).
'''



p = './w:r//w:t'
ReferenceOld_xpath = './w:r[w:rPr[w:rStyle[@w:val="ReferenceOld"]]]/w:t'
ReferenceNew_xpath = './w:r[w:rPr[w:rStyle[@w:val="ReferenceNew"]]]/w:t'
PhysicalDescription_xpath = './w:r[w:rPr[w:rStyle[@w:val="PhysicalDescription"]]][w:t]/w:t'
ContentDescription_xpath = './w:pPr[w:pStyle[@w:val="ContentDescription"]]/following-sibling::w:r/w:t'
title_xpath = './w:pPr[w:pStyle[@w:val="Title"]]/following-sibling::w:r/w:t'
date_xpath = './w:r[w:rPr[w:rStyle[@w:val="ItemDate"]]][w:t]/w:t'

In [65]:
table = pd.DataFrame()

ReferenceOld_dic = {}
ReferenceNew_dic = {}
title_dic = {}
ContentDescription_dic = {}
PhysicalDescription_dic = {}
date_dic = {}


for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    
    ##TODO: here we could add some contraints in case there is just old or new refernce (or we add it to the guidelines)
    
    check = para.xpath(ReferenceOld_xpath, namespaces=ns)
    if check:
        text = para.xpath(ReferenceOld_xpath, namespaces=ns)
        ReferenceOld = " ".join([t.text.strip() for t in text]).strip('\n')
    
    check = para.xpath(ReferenceNew_xpath, namespaces=ns)
    if check:
        text = para.xpath(ReferenceNew_xpath, namespaces=ns)
        ReferenceNew = " ".join([t.text.strip() for t in text]).strip('\n')

        
    
    check = para.xpath(title_xpath, namespaces=ns)
    if check:
        text = para.xpath(title_xpath, namespaces=ns)
        title = " ".join([t.text.strip() for t in text]).strip('\n')
    else: 
        title = ''
    
    check = para.xpath(PhysicalDescription_xpath, namespaces=ns)
    if PhysicalDescription_xpath:
        text = para.xpath(PhysicalDescription_xpath, namespaces=ns)
        PhysicalDescription = " ".join([t.text.strip() for t in text]).strip(' \n')
    else: 
        PhysicalDescription = ''
    
    check = para.xpath(ContentDescription_xpath, namespaces=ns)
    if ContentDescription_xpath:
        text = para.xpath(ContentDescription_xpath, namespaces=ns)
        ContentDescription = " ".join([t.text.strip() for t in text]).strip(' \n')
    else: 
        ContentDescription = ''
    
    
    check = para.xpath(date_xpath, namespaces=ns)
    if check:
        date = para.xpath(date_xpath, namespaces=ns)[0].text.strip()
    else:
        date = ''
        
        
        
    if ReferenceOld not in ReferenceOld_dic:
        ReferenceOld_dic[ReferenceOld] = ReferenceOld   
   

    if ReferenceOld not in ReferenceNew_dic:
        ReferenceNew_dic[ReferenceOld] = ReferenceNew   
        
        
         
    if ReferenceOld not in ContentDescription_dic:
        ContentDescription_dic[ReferenceOld] = ContentDescription   
    else:
        if ContentDescription!='':
            ContentDescription_dic[ReferenceOld]+="\n"+ContentDescription
        else:
            ContentDescription_dic[ReferenceOld]=ContentDescription_dic[ReferenceOld]
            
    
    if ReferenceOld not in title_dic:
        title_dic[ReferenceOld] = title
    else: 
        if title!='':
            title_dic[ReferenceOld]+="\n"+title
        else:
            title_dic[ReferenceOld]=title_dic[ReferenceOld]
            
        
    
    if ReferenceOld not in PhysicalDescription_dic:
        PhysicalDescription_dic[ReferenceOld] = PhysicalDescription
    else:
        if PhysicalDescription!='':
            PhysicalDescription_dic[ReferenceOld]+="\n"+PhysicalDescription
        else:
            PhysicalDescription_dic[ReferenceOld]=PhysicalDescription_dic[ReferenceOld]
    
    if ReferenceOld not in date_dic:
        date_dic[ReferenceOld] = date
    else: 
        if date!='':
            date_dic[ReferenceOld]+="\n"+date
        else:
            date_dic[ReferenceOld]=date_dic[ReferenceOld]
            
        
        
    
        

    

table = pd.DataFrame.from_dict({'ReferenceOld':pd.Series(ReferenceOld_dic),'ReferenceNew':pd.Series(ReferenceNew_dic),'Title':pd.Series(title_dic), 'PhysicalDescription':pd.Series(PhysicalDescription_dic), 'ContentDescription':pd.Series(ContentDescription_dic), 'Item date':pd.Series(date_dic)}
)
    
table = table.applymap(lambda x: x.strip('\n') if type(x)==str else x)

header_list = ['ReferenceOld', 'ReferenceNew', 'Title', 'PhysicalDescription','ContentDescription', 'Item date','Language','Creator']
table = table.reindex(columns = header_list) 

table.to_csv(file+".csv", encoding="utf-8") 



In [66]:
table

,ReferenceOld,ReferenceNew,Title,PhysicalDescription,ContentDescription,Item date,Language,Creator
5,5,MSS Eur F.1,[JOSIAH WEBBE?],39 x 25 cm. pp. 126.,"Verbael , uijt afgesonden en aengekomen brie...",,NaN,NaN
1,1,Mss. Eur. G. 1 .,"[“ Rec d from Exam rs Office"" 5 Oct. 1814.]\...","50 x 30 cm, pp. 264.","There is no general title, but the manuscript ...",1914 and 1919.,NaN,NaN
2,2,Mss. Eur. D. 2,A Decree of the Holy Congregation Generall for...,"30 x 18.5 cm, ' Foll . 3.","The watermarks are (a) Arms, Quarterly: 1st an...",,NaN,NaN
3,3,MSS. Eur. G. 2 .,Peticion of ye East India Company.,"44 x 32' cm. One sheet, framed and hung in the...",This document is reproduced (actual size) in R...,,NaN,NaN
4,4,MSS.Eur . D.3 .,"[Purchased 14 July 1916.]\n[ Batavia's , Statu...","33 x '21 cm. pp., vi, 200","This volume is lettered "" Batasia's Statut B...",,NaN,NaN


In [46]:
print(ReferenceNew_dic)

{'5': 'MSS Eur F.1 ', '1': 'Mss. Eur. G. 1 . ', '2 ': 'Mss. Eur. D. 2', '3 ': 'MSS. Eur. G. 2 .', '4': 'MSS.Eur . D.3 .'}


In [6]:
for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    
    
    check = para.xpath(ReferenceOld_xpath, namespaces=ns)
    if check:
        text = para.xpath(ReferenceOld_xpath, namespaces=ns)
        ReferenceOld = " ".join([t.text.strip() for t in text]).strip('\n')
        print(ReferenceOld)

1
2 
3 
4
5


In [ ]:
#find the initial position of each record

cat_idx_pos = []

for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
    if para.xpath(ReferenceOld_xpath, namespaces=ns):
        #print(i,para)
        cat_idx_pos.append(i)
print(cat_idx_pos)
#print(etree.tostring(cats[0], pretty_print=True))


In [ ]:
# create ranges 

records = []
for i,cat_id in enumerate(cat_idx_pos):
    if cat_id != cat_idx_pos[-1]:
        print
        idx = range(cat_id, cat_idx_pos[i+1])
        records.append([x for x in idx])
print(records)



In [ ]:
# create a catalogue including the relevant paras
record = []
catalogue_record_list = []

for x in records:
        for ix in x:
            catalogue_record = etree.Element("record")
            for i, para in enumerate(et.xpath('//w:p', namespaces=ns)):
                if i==ix:
                    catalogue_record.append(copy.deepcopy(para))
        catalogue_record_list.append(catalogue_record)
        
    
          
        #print(etree.tostring(catalogue_record, pretty_print=True))
        
len(catalogue_record_list)

In [ ]:

table_turkish = pd.DataFrame()
table = pd.DataFrame()

catalogue_record_list = catalogue_record_list[0]
for cat_rec in catalogue_record_list:
    for i, para in enumerate(cat_rec.xpath('./w:p', namespaces=ns)):

        check = para.xpath(ReferenceOld_xpath, namespaces=ns)
        if check:
            ReferenceOld = para.xpath(ReferenceOld_xpath, namespaces=ns)[0].text.strip()
        else:
            ReferenceOld = ''

        if para.xpath(title_xpath, namespaces=ns):
            title = para.xpath(title_xpath, namespaces=ns)[0].text.strip()
        else: 
            title = ''

        if para.xpath(PhysicalDescription_xpath, namespaces=ns):
            text = para.xpath(PhysicalDescription_xpath, namespaces=ns)
            PhysicalDescription = " ".join([t.text.strip() for t in text]).strip(' \n')
        else: 
            PhysicalDescription = ''

            
        if para.xpath(ContentDescription_xpath, namespaces=ns):
            text = para.xpath(ContentDescription_xpath, namespaces=ns)
            description = " ".join([t.text.strip() for t in text]).strip(' \n')
            pilcrow = """
                """
            description = re.sub(r"[\r\n]+", pilcrow, description)
        else:
            description = ''

        if para.xpath(date_xpath, namespaces=ns):
            date = para.xpath(date_xpath, namespaces=ns)[0].text.strip()
        else: 
            date = ''

        table = table.append({'ReferenceOld':ReferenceOld, 'title': title, 'PhysicalDescription': PhysicalDescription, 'description': description, 'date': date}, ignore_index=True)
        table.to_csv('test.csv', encoding="utf-8") 

In [ ]:
#approccio diverso


catalogue_record_list = catalogue_record_list[0]
for cat_rec in catalogue_record_list:
    for i, para in enumerate(cat_rec.xpath('./w:p', namespaces=ns)):
        catalogue = {}
        
        check = para.xpath(ReferenceOld_xpath, namespaces=ns)
        if check:
            ReferenceOld = para.xpath(ReferenceOld_xpath, namespaces=ns)[0].text.strip()
        else:
            ReferenceOld = ''

        if para.xpath(title_xpath, namespaces=ns):
            title = para.xpath(title_xpath, namespaces=ns)[0].text.strip()
        else: 
            title = ''

        if para.xpath(PhysicalDescription_xpath, namespaces=ns):
            text = para.xpath(PhysicalDescription_xpath, namespaces=ns)
            PhysicalDescription = " ".join([t.text.strip() for t in text]).strip(' \n')
        else: 
            PhysicalDescription = ''

            
        if para.xpath(ContentDescription_xpath, namespaces=ns):
            text = para.xpath(ContentDescription_xpath, namespaces=ns)
            description = " ".join([t.text.strip() for t in text]).strip(' \n')
            pilcrow = """
                """
            description = re.sub(r"[\r\n]+", pilcrow, description)
        else:
            description = ''

        if para.xpath(date_xpath, namespaces=ns):
            date = para.xpath(date_xpath, namespaces=ns)[0].text.strip()
        else: 
            date = ''

In [ ]:
for a in catalogue_record_list[0].xpath('w:p/w:r/w:t', namespaces=ns):
    print(a.text)

In [ ]:
catalogue_record_list[0].xpath('w:p/w:r', namespaces=ns)

In [ ]:
et.xpath('//w:p', namespaces=ns